In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
#import nglview as nv
#import MDAnalysis as mda
from numpy.linalg import norm
# https://userguide.mdanalysis.org/stable/examples/constructing_universe.html

Matplotlib is building the font cache; this may take a moment.


In [2]:
# put everything in a universe
#Nres = Npart
#resIndices = np.repeat(range(Nres), 1)
#segIndices = [0] * Nres
#sol = mda.Universe.empty(Npart,
#                         n_residues = Nres,
#                         atom_resindex = resIndices,
#                         residue_segindex=segIndices,
#                         trajectory = True)
#sol.add_TopologyAttr('name', ['A1']*Nres)
#coordinates = np.array([x,y,z]).T
#assert coordinates.shape == (Npart, 3)
#sol.atoms.positions = coordinates

In [3]:
def initialization(Npart, T, dt, Lx, Ly, Lz):
    
    xlo, xhi = -Lx/2, Lx/2
    ylo, yhi = -Ly/2, Ly/2
    zlo, zhi = -Lz/2, Lz/2  

    # initialise positions and velocities
    x, y, z = np.zeros((3,Npart))
    xm, ym, zm = np.zeros((3,Npart))
    vx, vy, vz = np.zeros((3,Npart)) 

    for N in range(Npart):
        x[N] = random.random()*Lx+xlo
        y[N] = random.random()*Ly+ylo
        z[N] = random.random()*Lz+zlo
    
    vcom = np.zeros(3)
    ekin = 0
    for N in range(Npart):
        vx[N] = random.random()-0.5
        vy[N] = random.random()-0.5
        vz[N] = random.random()-0.5
        vcom += vx[N], vy[N], vz[N]
        ekin += vx[N]**2 + vy[N]**2 + vz[N]**2

    vcom /= Npart
    ekin /= Npart
    sfac = np.sqrt(3*T/ekin)

    for N in range(Npart):
        vx[N] = (vx[N]-vcom[0])*sfac
        vy[N] = (vy[N]-vcom[1])*sfac
        vz[N] = (vz[N]-vcom[2])*sfac
        xm[N] = x[N] - vx[N]*dt
        ym[N] = y[N] - vy[N]*dt
        zm[N] = z[N] - vz[N]*dt

    return x, y, z, vx, vy, vz, xm, ym, zm

In [4]:
def evalforce(Npart, x, y, z, Lx, Ly, Lz, cutoff):

    forces = np.zeros((Npart,3))
    box = np.array([Lx,Ly,Lz])
    for Ni in range(Npart-1):
        posi = np.array([x[Ni],y[Ni],z[Ni]])
        for Nj in np.arange(Ni+1,Npart):
            posj = np.array([x[Nj],y[Nj],z[Nj]])
            dij = np.remainder(posi - posj + box/2., box) - box/2.
            rij = norm(dij)
            if rij < cutoff:
                LJf = 48*1/rij**2*1/rij**6*(1/rij**6-0.5)
                forces[Ni] += LJf*dij
                forces[Nj] -= LJf*dij
    return forces

In [5]:
def equationsmotions(x,y,z,forces):
    velx, vely, velz = 0, 0, 0
    velx2, vely2, velz2 = 0, 0, 0
    for Ni in range(Npart):
        xx = 2*x[Ni]-xm[Ni]+dt**2*forces[Ni][0]
        yy = 2*y[Ni]-ym[Ni]+dt**2*forces[Ni][1]
        zz = 2*z[Ni]-zm[Ni]+dt**2*forces[Ni][2]

        vx = (xx-xm[Ni]) / (2*dt)
        vy = (yy-ym[Ni]) / (2*dt)
        vz = (zz-zm[Ni]) / (2*dt)
        
        velx += vx
        vely += vy
        velz += vz

        velx2 += vx**2
        vely2 += vy**2
        velz2 += vz**2  
 
        xm[Ni] = x[Ni]
        ym[Ni] = y[Ni]
        zm[Ni] = z[Ni]
        
        x[Ni] = xx
        y[Ni] = yy
        z[Ni] = zz
        
    T = (velx2+vely2+velz2)/3/Npart
    etot = 0.5*(velx2+vely2+velz2)/Npart
    return x, y, z, T, etot

In [6]:
# choose number of particles
Npart = 50
# choose a temperature
T = 1.0
# choose a timestep
dt = 0.005
# set box size
Lx = 20
Ly = 20
Lz = 20
# choose cut-off
cutoff = 3

x, y, z, vx, vy, vz, xm, ym, zm = initialization(Npart, T, dt, Lx, Ly, Lz)
for t in range(1000):
    forces = evalforce(Npart, x, y, z, Lx, Ly, Lz, cutoff)
    x, y, z, T, etot = equationsmotions(x,y,z,forces)
    
    if t%10 == 0:
        if t == 0:
            trajectory = ''
        trajectory += str(len(x)) + '\ntitle\n'
        for i in range(len(x)):
            trajectory += ' '.join(['Ar',str(x[i]),str(y[i]),str(z[i]),'\n'])

<ipython-input-4-41f91bca389b>:12: RuntimeWarning: divide by zero encountered in double_scalars
  LJf = 48*1/rij**2*1/rij**6*(1/rij**6-0.5)
<ipython-input-4-41f91bca389b>:13: RuntimeWarning: invalid value encountered in multiply
  forces[Ni] += LJf*dij
<ipython-input-4-41f91bca389b>:14: RuntimeWarning: invalid value encountered in multiply
  forces[Nj] -= LJf*dij
<ipython-input-4-41f91bca389b>:9: RuntimeWarning: invalid value encountered in remainder
  dij = np.remainder(posi - posj + box/2., box) - box/2.


In [10]:
str(x[i])

'nan'

In [9]:
trajectory

'50\ntitle\nAr -1.7150338637832794 -0.7658232679534444 -7.882912654555537 \nAr -5.597684635752412 -1.4244712063638763 -2.948948675435973 \nAr 2.4602150327728993 3.9518793388088134 7.379683041257014 \nAr -1.5762694922786824 -1.169490217791376 9.887674294455376 \nAr 2.424182483350949 -3.2711289519270443 -4.921182792789721 \nAr 2.9996319220935006 -3.1436939651223494 -2.136617047029309 \nAr -6.413754524444771 6.577047860673 -4.286720525948201 \nAr 7.3262616183250335 6.409328371509833 8.510844221050135 \nAr 5.998817634542696 -6.181934210296255 -4.673391379467624 \nAr 16693.770872654473 16288.937638979793 -5655.708504570015 \nAr -9.740474978662565 -5.7491720921827225 0.09183713596123806 \nAr -2.934370105208764 1.7867636765525718 3.247286761579231 \nAr -0.7224116554809552 0.167038126356151 4.067940906937779 \nAr -3.5593771538888097 5.021634794700831 -9.109694308386437 \nAr 1.806137649112357 -1.8440449576180735 -2.6814964443286478 \nAr -16704.075632767686 -16307.329300762452 5659.724745335681 

In [7]:
import py3Dmol
view = py3Dmol.view()
view.addModelsAsFrames(trajectory,'xyz')
view.animate({'loop': 'forward', 'reps': 1})
view.setStyle({'sphere':{'radius': 0.5}})
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
trajectory

'50\ntitle\nAr -1.7150338637832794 -0.7658232679534444 -7.882912654555537 \nAr -5.597684635752412 -1.4244712063638763 -2.948948675435973 \nAr 2.4602150327728993 3.9518793388088134 7.379683041257014 \nAr -1.5762694922786824 -1.169490217791376 9.887674294455376 \nAr 2.424182483350949 -3.2711289519270443 -4.921182792789721 \nAr 2.9996319220935006 -3.1436939651223494 -2.136617047029309 \nAr -6.413754524444771 6.577047860673 -4.286720525948201 \nAr 7.3262616183250335 6.409328371509833 8.510844221050135 \nAr 5.998817634542696 -6.181934210296255 -4.673391379467624 \nAr 16693.770872654473 16288.937638979793 -5655.708504570015 \nAr -9.740474978662565 -5.7491720921827225 0.09183713596123806 \nAr -2.934370105208764 1.7867636765525718 3.247286761579231 \nAr -0.7224116554809552 0.167038126356151 4.067940906937779 \nAr -3.5593771538888097 5.021634794700831 -9.109694308386437 \nAr 1.806137649112357 -1.8440449576180735 -2.6814964443286478 \nAr -16704.075632767686 -16307.329300762452 5659.724745335681 